In [1]:
import requests


In [9]:
#r = requests.get('https://httpbin.org/basic-auth/user/pass')
r = requests.get('http://169.254.69.140:8080/?action=get&param=cnt', proxies = {'http': '169.254.191.30:8080'}, timeout = 5)
#s = session_for_src_addr('202.96.134.133')



ConnectTimeout: ignored

In [12]:
import urllib.request

def connect(host='http://169.254.69.140:8080/?action=get&param=cnt'):
    try:
        urllib.request.urlopen(host) #Python 3.x
        return True
    except:
        return False

# test
print( "connected" if connect() else "no internet!" )

no internet!


In [ ]:
%measures (with quCR) and displays S-value of CHSH-inequality
function [S,coinci,settings] = CHSH_quCR()
	ip='192.168.20.65'; % the ip address of the quCR in your local network
    inttime=1000; % integration time in ms
	pausetime=1; % wait time to allow for motor movement, increase when you notice your motors are too slow.

	%settings; optimize order to improve motor movement
	settings=[  0,  22.5;...
				0,  112.5;...
				90, 22.5;...
				90, 112.5;...
				45, 22.5;...
				45, 112.5;...
				135,22.5;...
				135,112.5;...
				0,  67.5;...
				0,  157.5;...
				90, 67.5;...
				90, 157.5;...
				45, 67.5;...
				45, 157.5;...
				135,67.5;...
				135,157.5];
			
	coinci=ones(16,1);
	
	set_integration(inttime,ip); %set the integration time for this experiment run, in ms. Min is 100, Max is 10000.
			
	   disp('Starting measurements');
	   for set=1:length(settings)
		  
		  disp(['Starting setting ' num2str(set) ...
			  ' (' num2str(settings(set,1)) ', ' num2str(settings(set,2)) ')' ]);		  
		  goto_setting(settings(set,1),settings(set,2),ip);%tell motors to move to the angles for new setting
		  pause(pausetime); %wait while motor is moving
		  [~,numofmeasureold]=measure(ip); %get measurement number
		  numofmeasure=numofmeasureold;
		  while numofmeasure==numofmeasureold
		  [coinci(set),numofmeasure]=measure(ip); %take first measurement after that
          pause(0.1);
		  end
		  
    end
	S=calc_S(coinci);
	disp(['------------- S-value: ' num2str(S) ' ---------------']);		
end

function set_integration(msTime, ip)
	%send the command to set the integration time to the quCR
	webread(['http://' ip ':8080/?action=set&param=int&value=' num2str(msTime)]);
end

function goto_setting(alpha,beta,ip)
	%send commands to set the motor position to quCR, readout is not important
	webread(['http://' ip ':8080/?action=set&param=pm1&value=' num2str(alpha)]);
	webread(['http://' ip ':8080/?action=set&param=pm2&value=' num2str(beta)]);
end

function [coinci,numofmeasure]=measure(ip)
	%read measurement string
	cntstr=webread(['http://' ip ':8080/?action=get&param=cnt']);

	%string parse
	startidx2=strfind(cntstr,'Measurements: ')+14;
	endidx2=strfind(cntstr,'<br>Integration ')-1;
	numofmeasure=str2double(cntstr(startidx2:endidx2));

	startidx=strfind(cntstr,'01: ')+4;
	endidx=strfind(cntstr,'<br>02: ')-1;
	coinci=str2double(cntstr(startidx:endidx));
end

function [S]=calc_S(coinci)
	%function to calculate the S Value from the vector of coincidence counts.
	E1=calc_E(coinci(1:4));
	E2=calc_E(coinci(5:8));
	E3=calc_E(coinci(9:12));
	E4=calc_E(coinci(13:16));
	S=E1+E2-E3+E4;
end

function [E]=calc_E(C)
	%function to calculate one of the four expectation values
    E=(C(1)-C(2)-C(3)+C(4))/sum(C);
end

In [11]:
import socket
IPaddress=socket.gethostbyname(socket.gethostname())
if IPaddress=="127.0.0.1":
    print("No internet, your localhost is "+ IPaddress)
else:
    print("Connected, with the IP address: "+ IPaddress )

Connected, with the IP address: 172.28.0.2
